# <h1 style="font-size:24px">Website traffic of Romanian media and advertising companies</h1>
--- 

## <h2 style="font-size:20px">**ABOUT THE DATA**</h2>
[BRAT](https://www.brat.ro/ce-este-brat) is a non-profit organization of the media and advertising industry. Their aim is to support this industry by establishing common methods and standards on how to measure the performance indicators of media products. One of these KPIs is site traffic, that is measured in the [SATI](https://www.brat.ro/sati) project. The findings are published each day on SATI's webpage, in the [Traffic results](https://www.brat.ro/sati/rezultate/type/site/page/1/c/all) menu. They have the possibility to export the whole dataset, not only the filtered table that can be seen on their webpage. With Inspect the URL for the generated Excel file can be found out, and it can be scaped with Python's requests library.

It is important to keep in mind, that site traffic data is available only for the day before the current date. Moreover, for those who are not a BRAT member, data can be accessed only for the last month.

### <h2 style="font-size:17px">**Data structure**</h2>
Downloaded data contains the following columns: 
* Categorie: the site's media category
* Site
* Sitecode
* Tip trafic: traffic type: desktop/laptop, mobile, mobile applications
* Editor site
* Contractor
* Regie de publicitate
* Afisari: Page Impression - a visitor displays a page on the site
* Vizite: Visit - a series of one or more impressions as a result of a visitor's request. A visit ends when the period between 2 consecutive impressions is longer than 30 minutes
* Clienti Unici: Unique user - a unique combination of IP address and other identifiers

From these only the category, traffic type, Contractor, page impression, visit and unique users will be used. Furthermore, the data is available for wordwide traffic and traffic from only Romania too.

## <h2 style="font-size:19px">**ABOUT THE NOTEBOOK**</h2>
This notebook is the main part of the project, where all things are summarized and put together from downloading the data until the part where it is put into InfluxDB. 

The data is scraped down into Excel format, so this can be imported directly into pandas - with the help of that the data processing part will be done. Fortunately, the data comes in a quite decent format, only the unneeded columns and empty rows should be deleted. Since it contains a total traffic row for all sites too, this row should have been eliminated too, as this is the sum of the trafic types. Lastly, because of InfluxDB's time series database nature, the dataframe's index should be of datetime type.

In [9]:
import data_processing_utils as utils #data download and normalization
import db_utils #utils to get db connection and write to db 
from datetime import datetime, timedelta

client = db_utils.get_data_frame_client('influxdb', 'site_traffic')

In [6]:
maxd = datetime.strptime("2022-04-05", "%Y-%m-%d")
d = yesterday_dt = datetime.today() - timedelta(days=1)

In [17]:
d

datetime.datetime(2022, 5, 3, 13, 3, 54, 273559)

In [16]:
# 

while d > maxd:
    d_str = d.strftime("%Y-%-m-%-d")
    print(d_str)
    #1 download and write WORLDWIDE TRAFFIC into db
    df = utils.get_normalized_resource(resource_url="https://www.brat.ro/sati/export-rezultate/export/xls/site/c/all/period_type/day/period_filter/"+ d_str +"/category/all/editor/all/order_by/name/order/desc/", file_path= "./data/" + d_str + ".xls", from_date=d, region_type='all')
    db_utils.write_data_to_db(client, df, 'traffic')

    #2  download and write ROMANIAN TRAFFIC into db
    df_ro = utils.get_normalized_resource(resource_url="https://www.brat.ro/sati/export-rezultate/export/xls/type/site-ro/c/custom/period_type/day/period_filter/"+ d_str +"/category/all/editor/all/order_by/name/order/asc/", file_path= "./data/" + d_str + "-ro.xls", from_date=d, region_type='ro')
    db_utils.write_data_to_db(client, df_ro, 'traffic')

    d = d - timedelta(days=1)
    
    break


2022-5-4
./data/2022-5-4.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
read file
normalized file
./data/2022-5-4-ro.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
read file
normalized file


In [37]:
d = 1


yesterday_dt = datetime.today() - timedelta(days=1)

In [5]:
d = datetime.today() - timedelta(days=1)
d_str = d.strftime("%Y-%#m-%#d")
d_str

'2022-05-04'

In [30]:
import data_processing_utils as utils
from datetime import datetime, timedelta

yesterday_dt = datetime.today() - timedelta(days=6)
yesterday_dt_str = yesterday_dt.strftime("%Y-%#m-%#d")
yesterday_dt_str

'2022-4-29'

In [33]:
#1 get the worldwide traffic
df = utils.get_normalized_resource(resource_url="https://www.brat.ro/sati/export-rezultate/export/xls/site/c/all/period_type/day/period_filter/"+ yesterday_dt_str +"/category/all/editor/all/order_by/name/order/desc/", file_path= "./data/" + yesterday_dt_str + ".xls", from_date=yesterday_dt, region_type='all')

#write to file


#2 get the RO traffic
df_ro = utils.get_normalized_resource(resource_url="https://www.brat.ro/sati/export-rezultate/export/xls/type/site-ro/c/custom/period_type/day/period_filter/"+ yesterday_dt_str +"/category/all/editor/all/order_by/name/order/asc/", file_path= "./data/" + yesterday_dt_str + "-ro.xls", from_date=yesterday_dt, region_type='ro')

#write to file


./data/2022-4-29.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
read file
normalized file


In [32]:
df

,category,site,traffic_type,SATI_contractor,value_type,field,traffic_region
timestamp,,,,,,,
2022-04-29 08:54:33.282851,Auto & moto,www.4tuning.ro,trafic desktop / laptop,Thinkdigital Internet & Advertising SA,page_impression,15541.0,all
2022-04-29 08:54:33.282851,Auto & moto,www.4tuning.ro,trafic desktop / laptop,Thinkdigital Internet & Advertising SA,visit,4797.0,all
2022-04-29 08:54:33.282851,Auto & moto,www.4tuning.ro,trafic desktop / laptop,Thinkdigital Internet & Advertising SA,unique_client,4289.0,all
2022-04-29 08:54:33.282851,Auto & moto,www.4tuning.ro,trafic mobile (tableta / smartphone),Thinkdigital Internet & Advertising SA,page_impression,65233.0,all
2022-04-29 08:54:33.282851,Auto & moto,www.4tuning.ro,trafic mobile (tableta / smartphone),Thinkdigital Internet & Advertising SA,visit,39520.0,all
...,...,...,...,...,...,...,...
2022-04-29 08:54:33.282851,Muzica & Audio,www.zutv.ro,trafic mobile (tableta / smartphone),Antena TV Group SA,visit,12220.0,all
2022-04-29 08:54:33.282851,Muzica & Audio,www.zutv.ro,trafic mobile (tableta / smartphone),Antena TV Group SA,unique_client,10744.0,all
2022-04-29 08:54:33.282851,Muzica & Audio,www.zutv.ro,aplicatii mobile,Antena TV Group SA,page_impression,0.0,all


OTHER

In [29]:
import imp
imp.reload(utils)

<module 'data_processing_utils' from 'c:\\EGYETEM\\3. ev\\2. felev\\Integralt rendszerek ERP\\egyeni\\src\\data_processing_utils.py'>

In [28]:
utils.normalize_df(df, 'all', yesterday_dt)

,category,site,traffic_type,SATI_contractor,value_type,field,traffic_region
timestamp,,,,,,,
2022-04-30 08:51:57.890095,Auto & moto,www.4tuning.ro,trafic desktop / laptop,Thinkdigital Internet & Advertising SA,page_impression,15541.0,all
2022-04-30 08:51:57.890095,Auto & moto,www.4tuning.ro,trafic desktop / laptop,Thinkdigital Internet & Advertising SA,visit,4797.0,all
2022-04-30 08:51:57.890095,Auto & moto,www.4tuning.ro,trafic desktop / laptop,Thinkdigital Internet & Advertising SA,unique_client,4289.0,all
2022-04-30 08:51:57.890095,Auto & moto,www.4tuning.ro,trafic mobile (tableta / smartphone),Thinkdigital Internet & Advertising SA,page_impression,65233.0,all
2022-04-30 08:51:57.890095,Auto & moto,www.4tuning.ro,trafic mobile (tableta / smartphone),Thinkdigital Internet & Advertising SA,visit,39520.0,all
...,...,...,...,...,...,...,...
2022-04-30 08:51:57.890095,Muzica & Audio,www.zutv.ro,trafic mobile (tableta / smartphone),Antena TV Group SA,visit,12220.0,all
2022-04-30 08:51:57.890095,Muzica & Audio,www.zutv.ro,trafic mobile (tableta / smartphone),Antena TV Group SA,unique_client,10744.0,all
2022-04-30 08:51:57.890095,Muzica & Audio,www.zutv.ro,aplicatii mobile,Antena TV Group SA,page_impression,0.0,all


In [9]:
URL = "https://www.brat.ro/sati/export-rezultate/export/xls/site/c/all/period_type/day/period_filter/"+ yesterday_dt_str +"/category/all/editor/all/order_by/name/order/desc/"
file_path = "./data/" + yesterday_dt_str + '.xls' #data should be downloaded in data folder with the name of the current date

# utils.download_resource(URL, file_path)

In [6]:
os.fspath(file_path)

'./data/2022-5-4.xls'

In [14]:
import pandas as pd

pd.read_excel(file_path, engine="xlrd")

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,Nr.,Categorie,Site,Sitecode,Tip trafic,Editor site,Contractor SATI,Regie de publicitate,Afisari,Vizite,Clienti Unici
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,Auto & moto,www.4tuning.ro,4tuningro,NaN,NaN,Thinkdigital Internet & Advertising SA,ThinkDigital,80785.0,44323.0,39260.0
2,NaN,Auto & moto,www.4tuning.ro,4tuningro,trafic desktop / laptop,NaN,Thinkdigital Internet & Advertising SA,ThinkDigital,15541.0,4797.0,4289.0
3,NaN,Auto & moto,www.4tuning.ro,4tuningro,trafic mobile (tableta / smartphone),NaN,Thinkdigital Internet & Advertising SA,ThinkDigital,65233.0,39520.0,34987.0
4,NaN,Auto & moto,www.4tuning.ro,4tuningro,aplicatii mobile,NaN,Thinkdigital Internet & Advertising SA,ThinkDigital,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
788,NaN,Stiri si analize locale,www.ziuaconstanta.ro,ziuaconstantaro,aplicatii mobile,NaN,Ziua Tomis SRL,NaN,0.0,0.0,0.0
789,198.0,Muzica & Audio,www.zutv.ro,zutvro,NaN,Antena TV Group SA,Antena TV Group SA,Intact Media,17285.0,12708.0,11186.0
790,NaN,Muzica & Audio,www.zutv.ro,zutvro,trafic desktop / laptop,Antena TV Group SA,Antena TV Group SA,Intact Media,972.0,487.0,441.0
791,NaN,Muzica & Audio,www.zutv.ro,zutvro,trafic mobile (tableta / smartphone),Antena TV Group SA,Antena TV Group SA,Intact Media,16312.0,12220.0,10744.0


In [117]:
yesterday_dt.strftime("%Y-%#m-%#d")

'2022-5-4'

In [9]:
import db_utils

client = db_utils.get_data_frame_client('influxdb', 'site_traffic') #site_traffic is the name of db
measurement = 'traffic'

In [ ]:
# client.query("show databases")

# client.create_database("site_traffic")
# client.create_retention_policy(dbname, '1000d', 1, default=True)

In [119]:
import data_processing_utils as utils

In [120]:
# get the date of yesterday in YYYY-M-D format (e.g. 2nd of May, 2022 => 2022-5-2) <--- brat.ro publishes info only for the day before current day

yesterday_dt_str

'2022-5-4'

In [125]:
"./data/" + yesterday_dt_str + ".xls"

'./data/2022-5-4.xls'

In [127]:
import imp
imp.reload(data_processing_utils)

<module 'data_processing_utils' from 'c:\\EGYETEM\\3. ev\\2. felev\\Integralt rendszerek ERP\\egyeni\\src\\data_processing_utils.py'>

In [141]:
file_path = "./data/" + str(yesterday_dt_str) + '.xls'

In [140]:
str(yesterday_dt_str)

'2022-5-4'

In [144]:
type(file_path) 

str

In [145]:
df_ro = pd.read_excel(str(file_path))

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [129]:
df = pd.read_excel("data/2022-5-4.xls", engine='xlrd')

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [128]:
# download and normalize data === worldwide traffic
df = utils.get_normalized_resource(resource_url="https://www.brat.ro/sati/export-rezultate/export/xls/site/c/all/period_type/day/period_filter/"+ yesterday_dt_str +"/category/all/editor/all/order_by/name/order/desc/", file_path="data/" + yesterday_dt_str + ".xls", from_date=yesterday_dt, region_type='all')

data/2022-5-4.xls


TypeError: expected str, bytes or os.PathLike object, not NoneType

# <b style = "font-size: 20px">I. Worldwide traffic</b>

## <p style="font-size: 17px">1.1 Download data from brat.ro</p>

In [5]:
URL = "https://www.brat.ro/sati/export-rezultate/export/xls/site/c/all/period_type/day/period_filter/"+ yesterday_dt_str +"/category/all/editor/all/order_by/name/order/desc/"
file_path = "./data/" + yesterday_dt_str + '.xls' #data should be downloaded in data folder with the name of the current date

# utils.download_resource(URL, file_path)

## <p style="font-size: 17px">1.2 Process data</p>

In [6]:
import pandas as pd

df = pd.read_excel(file_path, engine="xlrd")
df = utils.normalize_df(df.copy(deep=True)) 
df["traffic_region"] = "all" #column indicating that this traffic from all regions
df["timestamp"] = yesterday_dt #InfluxDB's "id", a datetime type value
df.set_index("timestamp", inplace = True)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


## <p style="font-size: 17px">1.3 Send data to InfluxDB</p>

In [12]:
import imp

imp.reload(db_utils)

NameError: name 'db_utils' is not defined

In [ ]:
db_utils.write_data_to_db(client, df, measurement)

# <b style = "font-size: 20px">II. Traffic exlusively from Romania</b>

## <p style="font-size: 17px">1.1 Download data from brat.ro</p>

In [10]:
URL_RO = "https://www.brat.ro/sati/export-rezultate/export/xls/type/site-ro/c/custom/period_type/day/period_filter/"+ yesterday_dt_str +"/category/all/editor/all/order_by/name/order/asc/"
file_path_ro = "./data/" + yesterday_dt_str + '-ro.xls' #data should be downloaded in data folder with the name of the current date

utils.download_resource(URL_RO, file_path_ro)

'ERROR, file already exists'

## <p style="font-size: 17px">1.2 Process data</p>

In [13]:
import pandas as pd

df_ro = pd.read_excel(file_path_ro, engine="xlrd")
df_ro = utils.normalize_df(df_ro.copy(deep=True))

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


## <p style="font-size: 17px">1.3 Send data to InfluxDB</p>

In [14]:
db_utils.write_data_to_db(client, df_ro, measurement)

Download old datasets

In [ ]:
# currently available datasets: until 4/4/2022

import data_processing_utils as utils
from datetime import datetime, timedelta

d = 2

yesterday_dt = datetime.today() - timedelta(days=d)
yesterday_dt_str = str(yesterday_dt.year) + "-" + str(yesterday_dt.month) + "-" + str(yesterday_dt.day)
yesterday_dt_str

#1 download and normalize 
URL = "https://www.brat.ro/sati/export-rezultate/export/xls/site/c/all/period_type/day/period_filter/"+ yesterday_dt_str +"/category/all/editor/all/order_by/name/order/desc/"


d+=1